# Guide RNA design for genes nearby given dbSNP positions  

In [1]:
from IPython.core.display import HTML
HTML('''
<style>
    div.prompt {display:none}
    div.cell{
        width:100%;
        margin-left:1%;
        margin-right:1%;
    }
</style>''')

# shift code cells
HTML('''
<style>
    div.input{
        width:100%;
        padding-left:2em;
        padding-right:0em;
    }
</style>''')
# code toggle button
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').show();
 } else {
 $('div.input').hide();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="See usage information at the end! Click here to toggle on/off the raw code. "></form>''')

In [2]:
%%html
<style>
.blueButton { background: cornflowerblue; foreground: white}
</style>

In [17]:
import subprocess

from Bio import Seq
import re
from itertools import product
import pandas as pd

import numpy as np

#  pip3.4  install sqlalchemy --user
# pip3.4  install pymysql --user 
from sqlalchemy import create_engine
import requests
import xml.etree.ElementTree as ET
from ipywidgets import *  
from IPython.display import display 

<pre>
Probaljuk meg RNA-Seq adattal kombinalni:

http://www.ebi.ac.uk/ena/data/view/SAMEA2242065

1. A bam jo lesz, indexelni kene, feltehetoen hg19-re illesztettek.
2. A bam-bol kene kiszedni az intervallumokat lefedettseggel / peak intenzitassal
3. match-elni a cds szekvenciakkal (ez elvileg megoldana az orientacio problemat, gen eleje/vege) azokat a geneket, amelyek expresszalnak.
4. talaltam egy gRNS tracket az UCSC-ben: Genes and Gene predictions / CRISPR
http://genome.ucsc.edu/cgi-bin/hgTrackUi?hgsid=567368705_4XmeseAHAa9w44Dpm54HewWCLbqD&c=chr3&g=crispr
5. A cds elso 200 bazisara eso 5 gRNS-t kene levalogatni. 2 algoritmussal perdiktaljak es rankorderelik a hasitasi hatasfok alapjan, 
A 60% felettieket (zolddel jelolve) kellene legyujteni, vagyis ebbol az elso otot.
</pre>


---
#### NEW 2016.11.29
---

- minden SNP-hez, (elsore probaljuk a rs6853490 SNPre es rs10486567  is ) jobbra-balra 10-10 exon, max 5M bp tavolsagon belul
- mindegyik exon max elso 200bp-ban (orientacio szerint) lekerjuk a gRNA track-bol, a monjuk 60% felettiek listajat
- illetve ugyanerre a szakaszra az atlagos RNAseq lefedettseget a BAM filebol
- ebbol lesz egy nSNP x 2 x 10 x nGRNA

#### Get  RNAseq for LNCaP

In [ ]:
OK, downloaded %%bash
cd /nagyvinyok/adat84/sotejedlik/csabai/ngs/gRNA
wget ftp://ftp.sra.ebi.ac.uk/vol1/ERA260/ERA260568/bam/LNCaP.tophat.bam

In [51]:
OK %%bash
cd /nagyvinyok/adat84/sotejedlik/csabai/ngs/gRNA
samtools index LNCaP.tophat.bam

#### Get coordinates for the SNPs

In [3]:
# Get genomic position from UCSC based on dbSNP rs number
# UCSC dbSNP table schema: http://ucscbrowser.genap.ca/cgi-bin/hgTables?db=hg38&hgta_group=varRep&hgta_track=snp142Common&hgta_table=snp142Common&hgta_doSchema=describe+table+schema
# dbSnpVersion=hg38.snp142 or similar
def getDbSnpPosition(rsIdList,dbSnpVersion):
    rsIdStr = ""
    for id in rsIdList:
        rsIdStr += "'" + id + "',"
    rsIdStr = rsIdStr[:-1]
    #print(rsIdStr)
    
    engine = create_engine("mysql+pymysql://genome@genome-mysql.cse.ucsc.edu")
    query = "select chrom, chromStart, chromEnd, name, observed from " + dbSnpVersion + " WHERE name IN (" + rsIdStr + ")"
    dbSNP_data =  pd.read_sql_query(query, engine)
    #chromRS.value = dbSNP_data.ix[0,0] # dbSNP_data['chrom']
    #chromRSStart.value = dbSNP_data.ix[0,1] #dbSNP_data['chromStart']
    #chromRSEnd.value = dbSNP_data.ix[0,2] #dbSNP_data['chromEnd']
    return dbSNP_data

In [4]:
rsList = ['rs6853490','rs10486567']

In [43]:
positions = getDbSnpPosition(rsList,'hg19.snp142')
positions.head(3)

,chrom,chromStart,chromEnd,name,observed
0,chr7,27976562,27976563,rs10486567,A/G
1,chr4,95544717,95544718,rs6853490,A/G


## Try to get crisprTargets track

In [49]:
engine = create_engine("mysql+pymysql://genome@genome-mysql.cse.ucsc.edu")
query = "select * from hg19.crisprTargets LIMIT 10 "
pd.read_sql_query(query, engine)

,fileName
0,/gbdb/hg19/crispr/crispr.bb


#### This is a problem: 
the crisprTargets table (see e.g. http://genome.ucsc.edu/cgi-bin/hgTables?db=hg38&hgta_group=genes&hgta_track=crisprTargets&hgta_table=crisprTargets&hgta_doSchema=describe+table+schema ) does not contain data, only the filename

## Get crisprRanges table

In [39]:
# given a list of dbSNP positions from result of the getDbSnpPosition() function, 
# returns the coordinates of first nGenes downstream an upstream crispr regions  
def getNearbyCrispsRanges(positions,genomeVersion, maxRange, nGenes):
    results = []
    engine = create_engine("mysql+pymysql://genome@genome-mysql.cse.ucsc.edu")

    
    for idx,pos in positions.iterrows():
        fromPos = str(max(pos['chromStart']-maxRange,0))
        toPos = str(pos['chromStart']+maxRange)
        chromPosStr = str(pos['chromStart'])
        
        query = "select * from " + genomeVersion + ".crisprRanges" + " WHERE chrom='"+ \
            str(pos['chrom']) + "' AND chromEnd BETWEEN " + fromPos + " AND " + chromPosStr  +\
            " ORDER BY " + chromPosStr + "-chromEnd LIMIT " + nGenes
        downstream = pd.read_sql_query(query, engine)
        
        query = "select * from " + genomeVersion + ".crisprRanges" + " WHERE chrom='"+ \
            str(pos['chrom']) + "' AND chromStart BETWEEN " + chromPosStr + " AND " + toPos  +\
            " ORDER BY chromStart-" + chromPosStr + " LIMIT " + nGenes
        upstream = pd.read_sql_query(query, engine)
        
        i = 0
        for gene in downstream.itertuples():
            chromStart = gene.chromStart
            chromEnd = gene.chromEnd
            pos1 = pos.copy()
            pos1['dir'] = -1
            pos1['gene_idx'] = i
            pos1['posStart'] = chromStart
            pos1['posEnd'] = chromEnd
            pos1['length'] = np.abs(pos1['posEnd'] - pos1['posStart'])
            
            #query = "select * from " + genomeVersion + ".crisprTargets" + " WHERE chrom='"+ \
            #str(pos['chrom']) + "' AND chromStart BETWEEN " + str(chromStart) + " AND " + str(chromEnd)  +\
            #" ORDER BY " + " chromStart " 
            
            #crisprTarget = pd.read_sql_query(query, engine)
            
            #for c in crisprTarget.itertuples():
            #    pos1 = np.column_stack([pos1,c])
            
            results.append(pos1)
            i = i + 1
            
        i = 0
        for gene in upstream.itertuples():
            chromStart = gene.chromStart
            chromEnd = gene.chromEnd
            pos1 = pos.copy()
            pos1['dir'] = 1
            pos1['gene_idx'] = i
            pos1['posStart'] = chromStart
            pos1['posEnd'] = chromEnd
            pos1['length'] = np.abs(pos1['posEnd'] - pos1['posStart'])
            
            #query = "select * from " + genomeVersion + ".crisprTargets" + " WHERE chrom='"+ \
            #str(pos['chrom']) + "' AND chromStart BETWEEN " + str(chromStart) + " AND " + str(chromEnd)  +\
            #" ORDER BY " + " chromStart " 
            
            #crisprTarget = pd.read_sql_query(query, engine)
            
            #for c in crisprTarget.itertuples():
            #    pos1 = np.column_stack([pos1,c])
            
            results.append(pos1)
            i = i + 1

    return results


In [40]:
#rsList0 = ['rs339331','rs1109815']
positions = getDbSnpPosition(rsList,'hg38.snp142')
maxRange = 5000000 # maximum +/- range to search
nGenes = '10' # number of upstream and downstream genes to return
xxx2 = getNearbyCrispsRanges(positions,'hg38',maxRange,nGenes)

In [68]:
crisprRanges = pd.DataFrame(xxx2)
crisprRanges

,chrom,chromStart,chromEnd,name,observed,dir,gene_idx,posStart,posEnd,length
0,chr7,27936943,27936944,rs10486567,A/G,-1,0,27914529,27915008,479
0,chr7,27936943,27936944,rs10486567,A/G,-1,1,27913191,27913653,462
0,chr7,27936943,27936944,rs10486567,A/G,-1,2,27895018,27895616,598
0,chr7,27936943,27936944,rs10486567,A/G,-1,3,27863644,27864280,636
0,chr7,27936943,27936944,rs10486567,A/G,-1,4,27846302,27846747,445
0,chr7,27936943,27936944,rs10486567,A/G,-1,5,27840497,27844764,4267
0,chr7,27936943,27936944,rs10486567,A/G,-1,6,27830372,27833351,2979
0,chr7,27936943,27936944,rs10486567,A/G,-1,7,27828427,27829965,1538
0,chr7,27936943,27936944,rs10486567,A/G,-1,8,27827537,27828020,483
0,chr7,27936943,27936944,rs10486567,A/G,-1,9,27816148,27817238,1090


#### Number of reads in:
/nagyvinyok/adat84/sotejedlik/csabai/ngs/gRNA/LNCaP.tophat.bam

In [80]:
for id,line in crisprRanges.iterrows():
    r = line['chrom'] + ':' + str(line['posStart'])+ '-' + str(line['posEnd'])
    p1 = subprocess.Popen(['/nagyvinyok/adat88/kozos/sotejedlik/usr/bin/samtools', 'view', '/nagyvinyok/adat84/sotejedlik/csabai/ngs/gRNA/LNCaP.tophat.bam', r], stdout=subprocess.PIPE)
    p2 = subprocess.Popen(['wc','-l'],stdin=p1.stdout, stdout=subprocess.PIPE)
    p1.stdout.close() 
    print(line['name'],r,int(p2.stdout.read()))
    p2.stdout.close()

rs10486567 chr7:27914529-27915008 0
rs10486567 chr7:27913191-27913653 0
rs10486567 chr7:27895018-27895616 0
rs10486567 chr7:27863644-27864280 0
rs10486567 chr7:27846302-27846747 0
rs10486567 chr7:27840497-27844764 0
rs10486567 chr7:27830372-27833351 0
rs10486567 chr7:27828427-27829965 0
rs10486567 chr7:27827537-27828020 0
rs10486567 chr7:27816148-27817238 0
rs10486567 chr7:27991708-27992452 0
rs10486567 chr7:28020352-28020999 0
rs10486567 chr7:28071417-28071884 0
rs10486567 chr7:28117912-28118419 0
rs10486567 chr7:28180121-28180943 0
rs10486567 chr7:28185469-28185952 0
rs10486567 chr7:28217042-28217549 0
rs10486567 chr7:28239683-28244117 0
rs10486567 chr7:28299120-28299641 0
rs10486567 chr7:28409134-28410970 0
rs6853490 chr4:94617803-94618391 0
rs6853490 chr4:94610868-94611400 0
rs6853490 chr4:94609960-94610504 0
rs6853490 chr4:94607858-94608378 0
rs6853490 chr4:94586788-94588416 0
rs6853490 chr4:94586207-94586644 0
rs6853490 chr4:94585364-94585937 0
rs6853490 chr4:94584200-94585203 0
